In [ ]:
import numpy as np 
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from collections import namedtuple


In [ ]:
############################TRACK CONSTRUCTION PROCESS########################
def track_accuracy(pred_track, truth_track):
  truths = []
  for i in range(len(pred_track)):
    for j in range (len(truth_track)):
      is_a_track = np.array_equal(pred_track[i] , truth_track[j], equal_nan=False)
      if is_a_track:
        truths.append(i)
        break
  acc = len(truths)/ len(truth_track)
  return acc, truths
def construct_graph(ids_array): 
  t0 = time.time()
  segment = ids_array.copy()
  graphs = []
  while len(segment) > 0:
    segment_list =[]
    no_more_conn = []
    for elem in segment:
      idx = np.where(ids_array[:,0] == elem[-1])
      connections = ids_array[idx]
      if len(connections) > 0:
        for conenction in connections:
          segment_list.append(np.unique(np.concatenate((elem,conenction))))
      else:
        no_more_conn.append(elem)
      
      segment = np.array(segment_list)
    graphs.append(np.array(no_more_conn))
  
  return graphs

In [ ]:
########COMPARISION LISTS###############
acc_seg = []
times_pred_seg = []
acc_track = []
times_pred_track = []


#########READ FILES################
for sec in range(8):
  df = pd.read_csv(f'/PATH_TO/GRAPH_BALANCED/Section_{sec}/event0.csv')

  #TRAINING
  for i in range(0,60):
    
    df2 = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/GRAPH_BALANCED/Section_{sec}/event{i}.csv')
    new_ids = np.arange(max(df.hit_id_2)+1,max(df.hit_id_2)+ max(df2.hit_id_2)+2)
    old_ids = np.arange(len(new_ids))
    new_h1 = []
    new_h2 = []
    for value in df2.hit_id_1.values:
        new_h1.append((new_ids[value]))
    for value in df2.hit_id_2.values:
        new_h2.append((new_ids[value]))
    df2['hit_id_1'] = new_h1
    df2['hit_id_2'] = new_h2
    df = pd.concat([df, df2])


  X_train = df[[ 'r_1', 'phi_1', 'z_1', 'ita_1','x_1','y_1','r_2', 'phi_2', 'z_2', 'ita_2','x_2','y_2','dphi', 'dz', 'dr', 'dita', 'dR','phi_slope', 'z0']]
  y_train = df.y

  #TESTING
  df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/GRAPH_BALANCED/Section_{sec}/event0.csv')
  for i in range(60,90):
    
    df2 = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/GRAPH_BALANCED/Section_{sec}/event{i}.csv')
    new_ids = np.arange(max(df.hit_id_2)+1,max(df.hit_id_2)+ max(df2.hit_id_2)+2)
    old_ids = np.arange(len(new_ids))
    new_h1 = []
    new_h2 = []
    for value in df2.hit_id_1.values:
        new_h1.append((new_ids[value]))
    for value in df2.hit_id_2.values:
        new_h2.append((new_ids[value]))
    df2['hit_id_1'] = new_h1
    df2['hit_id_2'] = new_h2
    df = pd.concat([df, df2])

  X_test = df[[ 'r_1', 'phi_1', 'z_1', 'ita_1','x_1','y_1','r_2', 'phi_2', 'z_2', 'ita_2','x_2','y_2','dphi', 'dz', 'dr', 'dita', 'dR','phi_slope', 'z0']]
  y_test = df.y
  
  ##############################XBG MODEL##############################################################
  
  model = MLPClassifier(hidden_layer_sizes=(15,15), random_state=1, max_iter= 200)
  
  model.fit(X_train, y_train)
 
  t0 = time.time()
  y_pred = model.predict(X_test)
  times_pred_seg.append(time.time()-t0)

  accuracy = accuracy_score(y_test, y_pred)
  acc_seg.append(accuracy)


########################BUIL TRACK####################################################################
  true_segments = df[['hit_id_1','hit_id_2']].iloc[np.where(y_test==True)].to_numpy()
  pred_segments = df[['hit_id_1','hit_id_2']].iloc[np.where(y_pred==True)].to_numpy()
  times_list =[]
  acc_list=[]
  time_graph = []
    
  true_tracks = construct_graph(true_segments)
  pred_tracks = construct_graph(pred_segments)
      
      
  accuracies = np.zeros(len(pred_tracks))
  correct_idx  = []
  predicted = np.zeros(len(pred_tracks))
  truth = np.zeros(len(pred_tracks))
  edge = np.zeros(len(pred_tracks))
  correct = np.zeros(len(pred_tracks))
  for i in range(len(pred_tracks)):
    predicted[i] = len(pred_tracks[i])
    truth[i] = len(true_tracks[i])
    edge[i] = i + 1
    acc, truths = track_accuracy(pred_tracks[i], true_tracks[i])
            
    correct[i] = len(truths)
    accuracies[i] = acc  
    correct_idx.append(truths)
  acc_track.append(accuracies[2:])     
 

In [ ]:
print(np.mean(np.array(acc_track).flatten()), np.std(np.array(acc_track).flatten()), np.mean(times_pred_seg), np.std(times_pred_seg))